In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import os
import requests
from bs4 import BeautifulSoup

import util as u

from glob import glob
from tqdm import tqdm_notebook as tqdm

pd.set_option('display.max_columns', 200)


In [4]:
#巡回先を作る
#枠とレースIDがあればマージできる？
result_infofile_list = glob("test/*/info_html/*") # 後々フォルダーが変わる
main_info_file = [x.replace("\\", "/") for x in result_infofile_list]

result_resultfile_list = glob("test/*/result_html/*") # 後々フォルダーが変わる
main_result_file = [x.replace("\\", "/") for x in result_resultfile_list]

target_info_list = [] #配列を空にする
target_result_list = [] #配列を空にする

match_up_info_list = [] #配列を空にする
match_up_result_list = [] #配列を空にする

#is_file = os.path.isfile("match_up.txt")

#if is_file:
#    with open("match_up.txt", "r") as tf:
#        match_up_list = tf.read().split(',')[:-1]
#    #途中から始めるためのコード書く
#    target_txts_list = set(result_file) - set(match_up_list)
#else:
#    target_txts_list = result_file

target_info_list = main_info_file[0:10]
target_result_list = main_result_file[0:1]

In [5]:
#データフレームを4個作る
main_info = pd.DataFrame(index=[])
main_info_dummy = pd.DataFrame(index=[])
main_result = pd.DataFrame(index=[])
main_result_dummy = pd.DataFrame(index=[])

#New_col = 'insert_1_1 insert_1_2 insert_2_1 insert_2_2 insert_3_1 insert_3_2 insert_4_1 insert_4_2 insert_5_1 insert_5_2 insert_6_1 insert_6_2 insert_7_1 insert_7_2 \
#            start_timing_1_1 start_timing_1_2 start_timing_2_1 start_timing_2_2 start_timing_3_1 start_timing_3_2 start_timing_4_1 start_timing_4_2 start_timing_5_1 start_timing_5_2 start_timing_6_1 start_timing_6_2 start_timing_7_1 start_timing_7_2 \
#            rank_1_1 rank_1_2 rank_2_1 rank_2_2 rank_3_1 rank_3_2 rank_4_1 rank_4_2 rank_5_1 rank_5_2 rank_6_1 rank_6_2 rank_7_1 rank_7_2'.split()

dfs_all = pd.DataFrame(index=[])
dfs_dummy = pd.DataFrame(index=[])

In [6]:
###後で消す↓###
main_info = pd.DataFrame(index=[])
main_info_dummy = pd.DataFrame(index=[])
###↑後で消す###

# info_htmlの加工をする
# race_id、枠、選手ID、級別、体重(直前情報から取る)、調整重量、F数、L数、平均ST、モーター2連、モーター3連、初日進入、
# 初日ST、初日成績、～ 、最終日、展示タイム、チルト、プロペラ、部品交換

for target_info in tqdm(target_info_list):
    for i in range(1, 7):
        target_info_day = target_info[-19:-5]
        Pit_No = f"{i:02}"
        
        #レース結果を保存したhtmlファイルとそのDF
        target_info_day = target_info[-19:-5]
        dfs = pd.read_html(target_info, encoding="utf-8")
        race_info1_df = dfs[0]
        
        #dfs[0]の整理
        df_player_info = u.split_inner_data(u.parse(race_info1_df), target_info_day)
        df_player_info = df_player_info.iloc[i-1]

        player_ID = df_player_info["pid"]

        #複勝率、三連率を計算
        player_win_rate = u.player_win_rate_cal(target_info_day, player_ID)
        se_player_win_rate = player_win_rate.iloc[0]

        #開催期間の艇番を表示
        df_bort_color = u.bort_color(target_info).iloc[i-1]
        #開催期間の艇番を表示
        df_open_grades = u.Open_Grades_sort(target_info).iloc[i-1]

        #結合する
        main_info_dummy = pd.concat([df_player_info, se_player_win_rate])
        main_info_dummy = pd.concat([main_info_dummy, df_bort_color])
        main_info_dummy = pd.concat([main_info_dummy, df_open_grades])
        
        main_info = main_info.append(main_info_dummy, ignore_index=True)

display(main_info.head(5))
#print(df_player_info[0])
#print(target_info)

#display(player_win_rate)
#display(dfs)
#display(dfs[1])
#display(dfs[2])


C:\Users\rstta\AppData\Local\Temp/ipykernel_16720/2633807899.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for target_info in tqdm(target_info_list):


  0%|          | 0/10 [00:00<?, ?it/s]

C:\Users\rstta\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2947: PerformanceWarning: indexing past lexsort depth may impact performance.
  return runner(coro)


,rid,en,pid,class,branch,weight,F,L,st,mnww,mnwww,idx,q_Total_2,q_Total_3,h_Total_2,h_Total_3,q_Venue_2,q_Venue_3,h_Venue_2,h_Venue_3,bort_color_1_1,bort_color_1_2,bort_color_2_1,bort_color_2_2,bort_color_3_1,bort_color_3_2,bort_color_4_1,bort_color_4_2,bort_color_5_1,bort_color_5_2,bort_color_6_1,bort_color_6_2,bort_color_7_1,bort_color_7_2,insert_1_1,insert_1_2,insert_2_1,insert_2_2,insert_3_1,insert_3_2,insert_4_1,insert_4_2,insert_5_1,insert_5_2,insert_6_1,insert_6_2,insert_7_1,insert_7_2,start_timing_1_1,start_timing_1_2,start_timing_2_1,start_timing_2_2,start_timing_3_1,start_timing_3_2,start_timing_4_1,start_timing_4_2,start_timing_5_1,start_timing_5_2,start_timing_6_1,start_timing_6_2,start_timing_7_1,start_timing_7_2,rank_1_1,rank_1_2,rank_2_1,rank_2_2,rank_3_1,rank_3_2,rank_4_1,rank_4_2,rank_5_1,rank_5_2,rank_6_1,rank_6_2,rank_7_1,rank_7_2
0,01_20140102_01,1.0,3115,B1,群馬,50.0,0,0,0.22,0.00,-,01_20140102_01_01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01_20140102_01,2.0,3560,B1,群馬,49.0,0,0,0.19,0.00,-,01_20140102_01_02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01_20140102_01,3.0,3376,B1,群馬,48.5,0,0,0.24,14.20,-,01_20140102_01_03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01_20140102_01,4.0,3532,A1,群馬,52.8,0,0,0.17,0.00,-,01_20140102_01_04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01_20140102_01,5.0,2808,B2,群馬,51.4,0,0,0.29,0.00,-,01_20140102_01_05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
weather_columns = "temp whether win_s win_a temp_w wave_h".split()
df_weather_conditions = pd.DataFrame(index=[], columns=weather_columns)


soup = BeautifulSoup(open("row_html/01/info_html/01_20140107_01.html", encoding="utf-8"), "lxml")

weather_bodys = soup.find_all("div", attrs={"class":"weather1_body"})[0]
weather_body = weather_bodys.find_all("div", attrs={"class":"weather1_bodyUnitLabel"})

weather_list = []

for i in range(len(weather_body)):
    weather_text = weather_body[i].text
    weather_text = weather_text.split("\n")
    del weather_text[0]
    del weather_text[-1]

    weather_list.append(weather_text)

#weather_wind_a = 
print(soup.select('[class^=weather1_bodyUnitImage]'))


In [12]:

print(weather_wind_a)


[]


In [9]:
df_weather_conditions.loc[0, "temp"] = weather_list[0][1].replace("℃", "")
df_weather_conditions.loc[0, "whether"] = weather_list[1][0]
df_weather_conditions.loc[0, "win_s"] = weather_list[2][1].replace("m", "")
df_weather_conditions.loc[0, "temp_w"] = weather_list[3][1].replace("℃", "")
df_weather_conditions.loc[0, "wave_h"] = weather_list[4][1].replace("cm", "")

display(df_weather_conditions)


,temp,whether,win_s,win_a,temp_w,wave_h
0,4.0,晴,2,NaN,4.0,1


In [10]:
[<div class="weather1_bodyUnitLabel">
<span class="weather1_bodyUnitLabelTitle">気温</span>
<span class="weather1_bodyUnitLabelData">4.0℃</span>
</div>, <div class="weather1_bodyUnitLabel">
<span class="weather1_bodyUnitLabelTitle">晴</span>
</div>, <div class="weather1_bodyUnitLabel">
<span class="weather1_bodyUnitLabelTitle">風速</span>
<span class="weather1_bodyUnitLabelData">2m</span>
</div>, <div class="weather1_bodyUnitLabel">
<span class="weather1_bodyUnitLabelTitle">水温</span>
<span class="weather1_bodyUnitLabelData">4.0℃</span>
</div>, <div class="weather1_bodyUnitLabel">
<span class="weather1_bodyUnitLabelTitle">波高</span>
<span class="weather1_bodyUnitLabelData">1cm</span>
</div>]

SyntaxError: invalid syntax (Temp/ipykernel_16720/3655555502.py, line 1)

### データが取れてないところからデータを取る処理を書く

### 現在開催されているデータを引っ張ってくる

### PyCaretってのを使ってみるよ